#### 1. Import dependencies

In [ ]:
%pip install stable-baselines3[extra]

In [ ]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

#### 2. Load environment

In [ ]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [ ]:
# Test environment
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    score = 0
    done = False

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode}, Score{score}")
env.close()

#### 3. Training an RL Model

In [ ]:
log_path = os.path.join('Training', 'Logs')
log_path

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
PPO??

In [ ]:
model.learn(total_timesteps=20000)

#### 4. Save and Reload Model

In [ ]:
PPO_Path = os.path.join('Training', 'Saved_Models', 'PPO_Model_Cartpole')
PPO_Path

In [ ]:
model.save(PPO_Path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_Path, env=env)

In [ ]:
model.learn(total_timesteps=1000)

#### 5. Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=5, render=True)

In [ ]:
env.close()

#### 6. Test Model

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    score = 0
    done = False

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode}, Score{score}")
env.close()

In [ ]:
obs = env.reset()
obs

In [ ]:
model.predict??

In [ ]:
action, _ = model.predict(obs)

In [ ]:
env.step(action)

#### 7. Viewing Logs in Tensorflow

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_1')
training_log_path

In [ ]:
!tensorboard --logdir={training_log_path}

#### 8. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
model_save_path = os.path.join('Training', 'Saved_Models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=model_save_path,
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

#### 9. Changing Policies

In [ ]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

#### 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)